In [7]:
# The hypothesis is that ASX codes are doing the same over years before covid and the yeaer that Covid affected Australia
# Here we have data from year 2014 to 2020 as before covid and also we are considering from 2020 to 2022 as Covid period
import scipy.stats as stats

# Read each sheet of the Excel file into separate DataFrames
file_path = "..\\DataCleaning\\resources\\complete_stock_data.csv"
# Read the CSV file into a DataFrame
final_df = pd.read_csv(file_path)
final_df.head()

,ASX Code,Security Group Code,Issuer Full Name,Product Description,Last Price($),Business Date,Year,Company Name,Sub-Industry,Splits,Dividends
0,1AD,EQY,ADALTA LIMITED ...,ORDINARY,0.280,20170630,2017,Adalta Limited Ordinary,Biotechnology & Medical Research,1.0,0.0
1,1AG,EQY,ALTERRA LIMITED ...,ORDINARY,0.024,20170630,2017,Alterra Limited Ordinary,Environmental Services & Equipment,1.0,0.0
2,3DP,EQY,POINTERRA LIMITED ...,ORDINARY,0.025,20170629,2017,Pointerra Limited Ordinary,Software,1.0,0.0
3,3PL,EQY,3P LEARNING LIMITED.. ...,ORDINARY,1.045,20170630,2017,3P Learning Limited. Ordinary,Software,1.0,0.0
4,4DS,EQY,4DS MEMORY LIMITED ...,ORDINARY,0.037,20170630,2017,4DS Memory Limited Ordinary,Computer Hardware,1.0,0.0


In [8]:
# Then, we will calculate the same analysis for that period.
during_covid_period = [2020, 2021, 2022] # Two years duration
before_covid_period = [2014, 2015, 2016, 2017, 2018, 2019, 2020] # Six years duration

# creating subsets of data for each period
during_covid_stocks_df = final_df[final_df['Year'].isin(during_covid_period)]
before_covid_stocks_df = final_df[final_df['Year'].isin(before_covid_period)]
during_covid_stocks_df["ASX Code"].nunique()

1842

In [9]:
before_covid_stocks_df["ASX Code"].nunique()

1403

In [10]:
# cleaning data
# we need to remove the codes that have only one entry so we can't calculate the performance 
# Group by 'ASXCode' and filter out groups with only one unique year
during_covid_stocks_df = during_covid_stocks_df.groupby('ASX Code').filter(lambda x: x['Year'].nunique() > 1)
before_covid_stocks_df = before_covid_stocks_df.groupby('ASX Code').filter(lambda x: x['Year'].nunique() > 1)
during_covid_stocks_df["ASX Code"].nunique()

1582

In [11]:
before_covid_stocks_df["ASX Code"].nunique()

1285

In [12]:
# Try to apply Splits on the prices of previous years for before covid period so that we can apply it on the price
# Step 1: Identify Split Events
bc_split_events = before_covid_stocks_df[before_covid_stocks_df['Splits'] > 1]  # Only select rows where Splits is not equal to 1

# Step 2: Adjust Historical Prices
for _, split_event in bc_split_events.iterrows():
    split_date = split_event['Year']
    split_ratio = split_event['Splits']
    asx_code = split_event['ASX Code']
    
    # Adjust historical prices for the ASX code before the split date
    before_covid_stocks_df.loc[(before_covid_stocks_df['ASX Code'] == asx_code) & (before_covid_stocks_df['Year'] < split_date), 'Last Price($)'] /= split_ratio

In [13]:
# Try to apply Splits on the prices of previous years for during covid period so that we can apply it on the price
# Step 1: Identify Split Events
dc_split_events = during_covid_stocks_df[during_covid_stocks_df['Splits'] > 1]  # Only select rows where Splits is not equal to 1

# Step 2: Adjust Historical Prices
for _, split_event in dc_split_events.iterrows():
    split_date = split_event['Year']
    split_ratio = split_event['Splits']
    asx_code = split_event['ASX Code']
    
    # Adjust historical prices for the ASX code before the split date
    during_covid_stocks_df.loc[(during_covid_stocks_df['ASX Code'] == asx_code) & (during_covid_stocks_df['Year'] < split_date), 'Last Price($)'] /= split_ratio

In [14]:
# convert the dividends from percent to dollar for before covid period
before_covid_stocks_df['Dividends'] = before_covid_stocks_df['Dividends'] * before_covid_stocks_df['Last Price($)'] /100

# Group data by 'ASX Code' and 'Year', and sort each group by 'Year' before finding the first and last prices
before_covid_first_last_prices = before_covid_stocks_df.groupby(['ASX Code', 'Year']).apply(lambda x: x.sort_values('Year')).reset_index(drop=True).groupby('ASX Code').agg({'Year': ['first', 'last'], 'Last Price($)': ['first', 'last'], 'Dividends': 'sum', 'Sub-Industry': 'last'}).reset_index()

# Rename the columns for clarity
before_covid_first_last_prices.columns = ['ASX Code', 'First Year', 'Last Year', 'First Price', 'Last Price', 'Total Dividends', 'Sub-Industry']

before_covid_first_last_prices


,ASX Code,First Year,Last Year,First Price,Last Price,Total Dividends,Sub-Industry
0,1AD,2017,2019,0.280,0.160,0.0,Biotechnology & Medical Research
1,1AG,2016,2019,0.052,0.042,0.0,Environmental Services & Equipment
2,3DP,2017,2019,0.025,0.043,0.0,Software
3,3PL,2015,2019,2.220,0.980,0.0,Software
4,4DS,2016,2019,0.025,0.061,0.0,Computer Hardware
...,...,...,...,...,...,...,...
1280,ZIP,2015,2019,0.770,0.012,0.0,Consumer Lending
1281,ZLD,2017,2019,0.074,0.040,0.0,Biotechnology & Medical Research
1282,ZMI,2017,2019,0.005,0.072,0.0,Integrated Mining
1283,ZNC,2014,2019,0.085,0.080,0.0,Integrated Mining


In [15]:
# convert the dividends from percent to dollar for during covid period
during_covid_stocks_df['Dividends'] = during_covid_stocks_df['Dividends'] * during_covid_stocks_df['Last Price($)'] /100

# Group data by 'ASX Code' and 'Year', and sort each group by 'Year' before finding the first and last prices
during_covid_first_last_prices = during_covid_stocks_df.groupby(['ASX Code', 'Year']).apply(lambda x: x.sort_values('Year')).reset_index(drop=True).groupby('ASX Code').agg({'Year': ['first', 'last'], 'Last Price($)': ['first', 'last'], 'Dividends': 'sum', 'Sub-Industry': 'last'}).reset_index()

# Rename the columns for clarity
during_covid_first_last_prices.columns = ['ASX Code', 'First Year', 'Last Year', 'First Price', 'Last Price', 'Total Dividends', 'Sub-Industry']

during_covid_first_last_prices


,ASX Code,First Year,Last Year,First Price,Last Price,Total Dividends,Sub-Industry
0,14D,2021,2022,0.095,0.075,0.0,Electrical Components & Equipment
1,1AD,2021,2022,0.125,0.055,0.0,Biotechnology & Medical Research
2,1AG,2021,2022,0.046,0.017,0.0,Environmental Services & Equipment
3,360,2021,2022,6.680,2.840,0.0,Software
4,3DA,2021,2022,0.575,0.165,0.0,Industrial Machinery & Equipment
...,...,...,...,...,...,...,...
1577,ZLD,2021,2022,0.043,0.970,0.0,Biotechnology & Medical Research
1578,ZMI,2021,2022,0.067,0.037,0.0,Integrated Mining
1579,ZMM,2021,2022,0.014,0.115,0.0,Electrical Components & Equipment
1580,ZNC,2021,2022,0.255,0.280,0.0,Integrated Mining


In [16]:
# Calculate total return (price appreciation + dividends) for before covid period
before_covid_first_last_prices['Total Return'] = before_covid_first_last_prices['Last Price'] - before_covid_first_last_prices['First Price'] + before_covid_first_last_prices['Total Dividends']

# Normalize total return
before_covid_first_last_prices['Normalized Total Return'] = (before_covid_first_last_prices['Total Return'] / before_covid_first_last_prices['First Price']) / 6 # the duration is 6 years so we need to devide it for normalization

# Sort ASX codes based on normalized total return
before_covid_sorted_data = before_covid_first_last_prices.sort_values(by='Normalized Total Return', ascending=False)

before_covid_sorted_data


,ASX Code,First Year,Last Year,First Price,Last Price,Total Dividends,Sub-Industry,Total Return,Normalized Total Return
771,MOT,2014,2019,0.002,2.040,0.0,Closed End Funds,2.038,169.833333
568,HLS,2014,2019,0.006,3.020,0.0,Healthcare Facilities & Services,3.014,83.722222
99,ANO,2014,2019,0.014,6.720,0.0,Personal Products,6.706,79.833333
855,NXS,2014,2019,0.013,4.110,0.0,Pharmaceuticals,4.097,52.525641
113,AQC,2014,2019,0.004,0.520,0.0,Coal,0.516,21.500000
...,...,...,...,...,...,...,...,...,...
1280,ZIP,2015,2019,0.770,0.012,0.0,Consumer Lending,-0.758,-0.164069
158,AVW,2017,2019,0.250,0.003,0.0,Gold,-0.247,-0.164667
942,PRL,2014,2018,0.175,0.002,0.0,Integrated Mining,-0.173,-0.164762
442,EPX,2014,2016,1.085,0.011,0.0,Software,-1.074,-0.164977


In [17]:
# Calculate total return (price appreciation + dividends) for during covid period
during_covid_first_last_prices['Total Return'] = during_covid_first_last_prices['Last Price'] - during_covid_first_last_prices['First Price'] + during_covid_first_last_prices['Total Dividends']

# Normalize total return
during_covid_first_last_prices['Normalized Total Return'] = (during_covid_first_last_prices['Total Return'] / during_covid_first_last_prices['First Price']) / 2 # the covid duration is 2 years so we need to devide it for normalization

# Sort ASX codes based on normalized total return
during_covid_sorted_data = during_covid_first_last_prices.sort_values(by='Normalized Total Return', ascending=False)

during_covid_sorted_data

,ASX Code,First Year,Last Year,First Price,Last Price,Total Dividends,Sub-Industry,Total Return,Normalized Total Return
1058,OAK,2021,2022,0.001,0.073,0.000000,Advanced Medical Equipment & Technology,0.072000,36.000000
936,MLS,2021,2022,0.002,0.050,0.000000,Integrated Mining,0.048000,12.000000
515,ELT,2021,2022,0.017,0.405,0.000000,Specialty Mining & Metals,0.388000,11.411765
1279,SCL,2021,2022,0.012,0.270,0.005074,Software,0.263074,10.961400
1577,ZLD,2021,2022,0.043,0.970,0.000000,Biotechnology & Medical Research,0.927000,10.779070
...,...,...,...,...,...,...,...,...,...
293,BWX,2021,2022,5.440,0.640,0.014976,Personal Products,-4.785024,-0.439800
653,GOLD,2021,2022,218.730,24.500,1.135654,Investment Management & Fund Operators,-193.094346,-0.441399
1528,WEC,2021,2022,0.130,0.010,0.003968,Coal,-0.116032,-0.446277
74,AHI,2021,2022,1.315,0.120,0.000000,Software,-1.195000,-0.454373


In [20]:
# Trying to run Paired t-test


# Each DataFrame should contain at least two columns: ASX codes and Normalized Total Return

# Merge the two DataFrames on ASX codes to ensure you have paired observations
paired_data = before_covid_sorted_data.merge(during_covid_sorted_data, on='ASX Code', suffixes=('_before', '_during'))

# Extract the normalized total return for each time period
normalized_total_return_before = paired_data['Normalized Total Return_before']
normalized_total_return_during = paired_data['Normalized Total Return_during']

# Perform the paired t-test
t_statistic, p_value = stats.ttest_rel(normalized_total_return_before, normalized_total_return_during)

# Output the results
print("Paired t-test results:")
print("T-statistic:", t_statistic)
print("P-value:", p_value)

Paired t-test results:
T-statistic: 3.020924086476264
P-value: 0.002572289363540979


In [21]:
# Independent samples t-test


# Each DataFrame should contain at least two columns: ASX codes and Normalized Total Return

# Extract the normalized total return for each time period
normalized_total_return_before = before_covid_sorted_data['Normalized Total Return']
normalized_total_return_during = during_covid_sorted_data['Normalized Total Return']

# Perform the independent samples t-test
t_statistic, p_value = stats.ttest_ind(normalized_total_return_before, normalized_total_return_during)

# Output the results
print("Independent samples t-test results:")
print("T-statistic:", t_statistic)
print("P-value:", p_value)


Independent samples t-test results:
T-statistic: 3.4477242598284685
P-value: 0.0005734816313346263
